<a href="https://colab.research.google.com/github/Howuhh/dul_2021/blob/hw9/Homework/hw9/Homework9_ebm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!if [ -d dul_2021 ]; then rm -Rf dul_2021; fi
!git clone https://github.com/GrigoryBartosh/dul_2021
!pip install ./dul_2021

Cloning into 'dul_2021'...
remote: Enumerating objects: 303, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 303 (delta 76), reused 68 (delta 58), pack-reused 171
Receiving objects: 100% (303/303), 54.22 MiB | 14.46 MiB/s, done.
Resolving deltas: 100% (137/137), done.
Processing ./dul_2021
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for dul-2021: filename=dul_2021-0.1.0-py3-none-any.whl size=24553 sha256=9112f87051251af4b9ab7eb35c2de88c42d0f93620b76cbcae34e1c49b9086eb
  Stored in directory: /tmp/pip-ephem-wheel-cache-slsibldz/wheels/55/59/29/0fb1c6356521577

In [2]:
from dul_2021.utils.hw9_utils import *

# Question 1: EBM on MNIST

Here we will implement EBM on MNIST data. 

* **Architecture.** You may experiment with different hyperparameters and architecture designs, but 
the following designs for the EBM architecture may be useful.

```
    Conv2d(1, 16, 5, 2, 4), 
    Swish(),
    Conv2d(16, 32, 3, 2, 1),
    Swish(), 
    Conv2d(32, 64, 3, 2, 1), 
    Swish(), 
    Conv2d(64, 64, 3, 2, 1), 
    Swish(),
    Flatten(),
    Linear(256, 64), 
    Swish(),
    Linear(64, 1)
```


Where `swish(x) = σ(x)x`. You **should** use activation with non-sparse gradients for better MCMC convergence.

* **Buffer.** To use the contrastive divergence objective, we need to generate samples during training. Because of high dimension of image space we need a lot of iterations in MCMC to obtain reasonable samples. We can reduce sampling cost with buffer trick. The idea of this trick it to re-use previous samples as starting points for MCMC (since distribuionts are close). In our implementation, we initialize 95% of starting points with previous samples and 5% with random noise from -1 to 1. 

* **MCMC.** During MCMC procedure you only need gradients wrt input. You can find it helpful to disable gradients of your model during MCMC. Since all images are in [-1, 1] you should clamp all resulting points during MCMC step. We also clamp gradients to [-0.03, 0.03].
    ```
    for p in model.parameters():
        p.requires_grad = False
    ```

* **Regularizations.**

 You should add small noise to real data otherwise ebm will focus on black areas of real images won't train.

 Add l2 regularaztion to ebm outputs on real and fake data. Otherwise, the output values will fluctuate in a very large range (because energy is invariant to scalar shift).

* **Hyperparameters**
    * Max buffer size - 8192
    * MCMC step size - 10
    * MCMC # step - 60
    * MCMC noise - N(0, 0.005)
    * Noise to data - N(0, 0.005)
    * l2 reg weight - 0.1
    * batch_size - 128
    * use Adam with lr=1e-3 and betas=(0, 0.999)
    * you can exponential scheduler
    * 20 epochs should be enough (~1 hour on gpu)



**You will provide the following deliverables**


1.   Over the course of training, record the regularization and contrastive losses per batch.
2. 100 samples from your trained EBM

In [20]:
import math
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import Adam

from torch.utils.data import DataLoader
from tqdm.auto import tqdm, trange

device = "cuda" if torch.cuda.is_available() else "cpu"

class Buffer:
    def __init__(self, buffer_size):
        self.buffer = torch.FloatTensor(buffer_size, 1, 28, 28).uniform_(-1.0, 1.0)

        self.buffer_size = buffer_size
        self.counter = 0
        self.real_size = 0

    def add(self, samples):
        n = samples.shape[0]

        if self.counter + n <= self.buffer_size:
            self.buffer[self.counter:self.counter + n] = samples
        else:
            reminder = self.buffer_size - self.counter
            self.buffer[-reminder:] = samples[:reminder]
            self.buffer[:reminder] = samples[reminder:]

        self.counter = (self.counter + n) % self.buffer_size
        self.real_size = min(self.real_size + 1, self.buffer_size)

    def sample(self, num_samples, limit_to_size=True):
        if limit_to_size:
            real_size = min(self.real_size, self.buffer_size)
            idxs = torch.randint(0, real_size, size=(num_samples,))
        else:
            idxs = torch.randint(0, self.buffer_size, size=(num_samples,))
        
        return self.buffer[idxs]

class EBM(nn.Module):
    def __init__(self, buffer_size=1024, reg_coef=0.1):
        super().__init__()
        self.energy = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=2, padding=4), 
            nn.SiLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.SiLU(), 
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1), 
            nn.SiLU(), 
            nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1), 
            nn.SiLU(),
            nn.Flatten(),
            nn.Linear(256, 64), 
            nn.SiLU(),
            nn.Linear(64, 1)
        )
        self.reg_coef = reg_coef
        self.buffer = Buffer(buffer_size)

    def require_model_grads(self, option=True):
        for p in self.energy.parameters():
            p.requires_grad = option

    def sample_mcmc(self, num_samples):
        self.require_model_grads(False)
        buffer_num_samples = int(num_samples * 0.95)

        eps_init = 10.0
        samples = torch.vstack([
            self.buffer.sample(buffer_num_samples, limit_to_size=False),
            torch.FloatTensor(num_samples - buffer_num_samples, 1, 28, 28).uniform_(-1, 1)
        ]).to(device)

        samples.requires_grad = True
        
        for step in range(60):
            z = torch.randn_like(samples) * math.sqrt(0.005)
            eps = eps_init - (eps_init * step) / 60
            
            grad_logp = torch.clip(torch.autograd.grad(self.energy(samples).mean(), samples)[0], -0.03, 0.03)
            samples = torch.clip(samples + math.sqrt(2 * eps) * z - eps * grad_logp, -1, 1)

        # adding to the buffer
        self.buffer.add(samples.detach().cpu())

        self.require_model_grads(True)
        return samples

    def _step(self, real_samples):
        real_samples = real_samples + (torch.randn_like(real_samples) * math.sqrt(0.005))
        model_samples = self.sample_mcmc(real_samples.shape[0]).to(device)

        model_energy, real_energy = self.energy(model_samples), self.energy(real_samples)

        contr_loss = (real_energy - model_energy).mean()
        reg_loss = (model_energy ** 2 + real_energy ** 2).mean()

        loss = contr_loss + self.reg_coef * reg_loss

        return loss, (contr_loss, reg_loss)

    def fit(self, train_dataloader, epochs=20, lr=1e-3):
        train_losses = []

        optim = Adam(self.energy.parameters(), lr=lr, betas=(0.0, 0.999))

        for epoch in trange(epochs, desc="Training"):
            for batch, labels in tqdm(train_dataloader, desc="Epoch", leave=False):
                batch = batch.to(device)

                loss, (contr_loss, reg_loss) = self._step(batch)

                optim.zero_grad()
                loss.backward()
                optim.step()

                train_losses.append([contr_loss.item(), reg_loss.item()])

        return np.array(train_losses)

In [21]:
def q1(train_data):
    """
    train_data: An (n_train, 1, 28, 28) torchvision dataset of MNIST images with values from -1 to 1

    Returns
    - a (# of training iterations, ) numpy array full of contrastive losses on each iteration
    - a (# of training iterations, ) numpy array full of regularization losses on each iteration
    - a (100, 28, 28, 1) numpy array of 100 samples from ebm model
    """
    dataloader = DataLoader(train_data, batch_size=128, pin_memory=True, shuffle=True)
    
    model = EBM(buffer_size=8192, reg_coef=0.1).to(device)
    losses = model.fit(dataloader, epochs=1, lr=1e-3)

    samples = model.sample_mcmc(100).detach().cpu().numpy()

    return losses[:, 0], losses[:, 1], samples

In [ ]:
q1_results(q1)

Training:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch:   0%|          | 0/469 [00:00<?, ?it/s]

# Question 2: EBM conditional generation

Here we will train EBM jointly with classifier to generate objects conditioned on class labels. ([original paper](https://arxiv.org/pdf/1912.03263.pdf)).

* **Theory**. Here we will work with joint distribution on x (data point) and y (label). We will parametrize probability of the pair as follows

```
p_θ(x, y) = exp(f_θ(x)[y]) / Z(θ)
```
Where `f(x)[y]` is an y-th output of the classifier model and `Z(θ)` is a normalization constant.

We will optimize log-likelihood of train pair:

```
log p(x, y) = log p(x) + log p(y|x)
```

Second term corresponds to normal classification loss and we will optimize it with cross entropy. We will optimize first tirm via contrastive divergence. The energy of can be from classifire output : `E(x)=-log ∑ exp(f_θ(x)[y])`.

* **Architecture.** We will work with smile dataset and you can use architecture from practice.

* **Hyperparameters**
    * Max buffer size - 8192
    * MCMC step size - 0.1
    * MCMC # step - 500 (during final sampling you can increase it)
    * MCMC noise - N(0, 0.005)
    * Noise to data - None
    * l2 reg weight - 0.1
    * batch_size - 128
    * use Adam with lr=1e-3 and betas=(0, 0.999)
    * you can use exponential scheduler
    * 100 epochs should be enough (~10 minutes on cpu)



**You will provide the following deliverables**


1.   Over the course of training, record loss per batch.
3. 300 samples from your trained EBM. First 100 samples should be generated for label 0, next 100 for label 1, and last 100 for label 2

In [ ]:
def q2(train_data, train_labels):
    """
    train_data: An (n_train, 2) np.array of binary points
    train_labels: An (n_train, ) np.array labels (3 classes)
    Returns
    - a (# of training iterations, ) numpy array full of losses on each iteration
    - a (300, 2) numpy array of 300 samples from ebm model (0-99 with label 0; 100-199 with label 1; 200-300 with label 2)
    """

In [ ]:
q2_results(q2)

# Bonus : Conditional generation with EBM and classifier.

Here you need to generate points conditioned on class on smile dataset with independently trained ebm and classifier. For generation use MCMC with

```
log p(x|y) ~ log p(x) + log p(y|x)
```

Where first term is `-E(x)` and second is obtained via backprop from trained classifier.

In [ ]:
def b(train_data, train_labels):
    """
    train_data: An (n_train, 2) np.array of binary points
    train_labels: An (n_train, ) np.array labels (3 classes)
    Returns
    - a (# of training iterations, ) numpy array full of losses of classfier on each iteration
    - a (# of training iterations, ) numpy array full of losses of ebm on each iteration
    - a (300, 2) numpy array of 300 samples from ebm model (0-99 with label 0; 100-199 with label 1; 200-300 with label 2)
    """

In [ ]:
b_results(b)